# Pipeline built-in factors

本文將介紹 zipline 內建的因子。

In [1]:
import pandas as pd 
import numpy as np 
import os 

os.environ['TEJAPI_KEY'] = "your key" 
os.environ['TEJAPI_BASE'] = "https://api.tej.com.tw"

os.environ['mdate'] = '20150101 20230702'
os.environ['ticker'] = "2330 IR0001"
!zipline ingest -b tquant

Merging daily equity files:


[2023-12-21 02:25:42.322694] INFO: zipline.data.bundles.core: Ingesting tquant.
[2023-12-21 02:25:44.519796] INFO: zipline.data.bundles.core: Ingest tquant successfully.


<span id="menu"></span>
## 選單
* [AverageDollarVolume](#AverageDollarVolume)
* [BollingerBands](#BollingerBands)
* [DailyReturns](#DailyReturns)
* [SimpleMovingAverage](#SimpleMovingAverage)
* [LinearWeightedMovingAverage](#LinearWeightedMovingAverage)
* [ExponentialWeightedMovingAverage](#ExponentialWeightedMovingAverage)
* [ExponentialWeightedMovingStdDev](#ExponentialWeightedMovingStdDev)
* [Latest](#Latest)
* [MaxDrawdown](#MaxDrawdown) 
* [Returns](#Returns)
* [RollingPearson](#RollingPearson)
* [RollingLinearRegressionOfReturns](#RollingLinearRegressionOfReturns)
* [RollingSpearmanOfReturns](#RollingSpearmanOfReturns)
* [SimpleBeta](#SimpleBeta)
* [RSI](#RSI)
* [VWAP](#VWAP)
* [WeightedAverageValue](#WeightedAverageValue)
* [PercentChange](#PercentChange)
* [PeerCount](#PeerCount)
* [RateOfChangePercentage](#RateOfChangePercentage)
* [Aroon](#Aroon)
* [FastStochasticOscillator](#FastStochasticOscillator)
* [TrueRange](#TrueRange)
* [IchimokuKinkoHyo](#IchimokuKinkoHyo)

In [2]:
from zipline.TQresearch.tej_pipeline import run_pipeline   
from zipline.pipeline import Pipeline
from zipline.pipeline.data import TWEquityPricing

start_time = pd.Timestamp("2018-02-02", tz="UTC")
end_time = pd.Timestamp("2022-07-02", tz="UTC")

<span id="AverageDollarVolume"></span>
### zipline.pipeline.factors.AverageDollarVolume

計算當日以前 n 天的資產平均價值，$ 每日平均價值 = 成交量 \times 價格$。  

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_, optional
        所欲計算之價格資料，預設 = EquityPricing.close。
* window_length: _int_
        決定 n 天。
        
[回到選單](#menu)

In [3]:
from zipline.pipeline.factors import AverageDollarVolume
def make_pipeline():
    return Pipeline(
        columns = {
            "avg_dollar_volume": AverageDollarVolume(
                inputs = [TWEquityPricing.close, TWEquityPricing.volume],
                window_length = 10
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

avg_dollar_volume
2018-02-02 00:00:00+00:00 Equity(0 [2330])         1.065447e+10
                          Equity(1 [IR0001])       4.975421e+13
2018-02-05 00:00:00+00:00 Equity(0 [2330])         9.914759e+09
                          Equity(1 [IR0001])       4.839457e+13
2018-02-06 00:00:00+00:00 Equity(0 [2330])         9.892107e+09
...                                                         ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])       7.144668e+13
2022-06-30 00:00:00+00:00 Equity(0 [2330])         1.734140e+10
                          Equity(1 [IR0001])       7.240678e+13
2022-07-01 00:00:00+00:00 Equity(0 [2330])         1.809191e+10
                          Equity(1 [IR0001])       7.315819e+13

[2146 rows x 1 columns]

<span id="BollingerBands"></span>
### zipline.pipeline.factors.BollingerBands

布林通道利用統計學概念，創造上、中、下三軌，中軌為 n 日的移動平均線，上下兩軌分別為中軌 $\pm m$ 個標準差。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_, optional
        所欲計算之價格資料，預設 = EquityPricing.close。
* window_length: _int_
        決定 n 天。
* k: _int_
        決定 m 個標準差。
        
#### Note:
不推薦以下使用方式。這種方式會產出帶有 tuple 的 DataFrame，在某些 pandas 版本（例如：1.5.3）下會出現`ValueError: no types given`的錯誤。

```python
def make_pipeline():
    return Pipeline(
        columns = {
            "bbands": BollingerBands(
                inputs = [TWEquityPricing.close], 
                window_length = 14,
                k=1.5
            )
        }
    )
```

[回到選單](#menu)

In [4]:
from zipline.pipeline.factors import BollingerBands
def make_pipeline():
    upper = BollingerBands(inputs = [TWEquityPricing.close], window_length = 14, k = 1.5).upper
    middle = BollingerBands(inputs = [TWEquityPricing.close], window_length = 14, k = 1.5).middle
    lower = BollingerBands(inputs = [TWEquityPricing.close], window_length = 14, k = 1.5).lower
    return Pipeline(
        columns = {
            "upper_bound":upper,
            "middle":middle,
            "lower_bound":lower
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

upper_bound        middle  \
2018-02-02 00:00:00+00:00 Equity(0 [2330])      265.252912    253.642857   
                          Equity(1 [IR0001])  19279.701951  19052.490000   
2018-02-05 00:00:00+00:00 Equity(0 [2330])      265.332723    255.035714   
                          Equity(1 [IR0001])  19268.235679  19073.285000   
2018-02-06 00:00:00+00:00 Equity(0 [2330])      264.351390    255.928571   
...                                                    ...           ...   
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  32611.739151  31462.749286   
2022-06-30 00:00:00+00:00 Equity(0 [2330])      517.756648    501.594794   
                          Equity(1 [IR0001])  32256.623293  31296.089286   
2022-07-01 00:00:00+00:00 Equity(0 [2330])      513.161348    497.942184   
                          Equity(1 [IR0001])  31978.169241  31095.972143   

                                               lower_bound  
2018-02-02 00:00:00+00:00 Equity(0 [2330])      242.032803  
                          Equity(1 [IR0001])  18825.278049  
2018-02-05 00:00:00+00:00 Equity(0 [2330])      244.738706  
                          Equity(1 [IR0001])  18878.334321  
2018-02-06 00:00:00+00:00 Equity(0 [2330])      247.505753  
...                                                    ...  
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  30313.759420  
2022-06-30 00:00:00+00:00 Equity(0 [2330])      485.432939  
                          Equity(1 [IR0001])  30335.555279  
2022-07-01 00:00:00+00:00 Equity(0 [2330])      482.723019  
                          Equity(1 [IR0001])  30213.775044  

[2146 rows x 3 columns]

<span id="DailyReturns"></span>
    
### zipline.pipeline.factors.DailyReturns

計算日報酬率。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_, optional
        所欲計算之價格資料，預設 = EquityPricing.close。
        
[回到選單](#menu)

In [5]:
from zipline.pipeline.factors import DailyReturns
def make_pipeline():
    return Pipeline(
        columns = {
            "Daily Return": DailyReturns(
                inputs = [TWEquityPricing.close]
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

Daily Return
2018-02-02 00:00:00+00:00 Equity(0 [2330])        0.017647
                          Equity(1 [IR0001])      0.005086
2018-02-05 00:00:00+00:00 Equity(0 [2330])        0.000000
                          Equity(1 [IR0001])     -0.003049
2018-02-06 00:00:00+00:00 Equity(0 [2330])       -0.025048
...                                                    ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])     -0.005615
2022-06-30 00:00:00+00:00 Equity(0 [2330])       -0.013065
                          Equity(1 [IR0001])     -0.009731
2022-07-01 00:00:00+00:00 Equity(0 [2330])       -0.030550
                          Equity(1 [IR0001])     -0.025612

[2146 rows x 1 columns]

<span id="SimpleMovingAverage"></span>

### zipline.pipeline.factors.SimpleMovingAverage

計算 n 日的簡單移動平均。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        所欲計算之資料。
* window_length: _int_
        決定 n 日。
        
[回到選單](#menu
)

In [6]:
from zipline.pipeline.factors import SimpleMovingAverage
def make_pipeline():
    return Pipeline(
        columns = {
            "SMA": SimpleMovingAverage(
                inputs = [TWEquityPricing.close], 
                window_length = 10
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

SMA
2018-02-02 00:00:00+00:00 Equity(0 [2330])      258.000
                          Equity(1 [IR0001])  19131.611
2018-02-05 00:00:00+00:00 Equity(0 [2330])      258.400
                          Equity(1 [IR0001])  19127.393
2018-02-06 00:00:00+00:00 Equity(0 [2330])      257.550
...                                                 ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  31072.561
2022-06-30 00:00:00+00:00 Equity(0 [2330])      496.550
                          Equity(1 [IR0001])  30962.418
2022-07-01 00:00:00+00:00 Equity(0 [2330])      493.350
                          Equity(1 [IR0001])  30799.546

[2146 rows x 1 columns]

<span id="LinearWeightedMovingAverage"></span>
### zipline.pipeline.factors.LinearWeightedMovingAverage

計算 n 日的線性加權移動平均，計算方法: 

$$
\frac{\sum_{i=1}^{n} i \times x_{i}}{\sum_{i=1}^{n} i}
$$

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        所欲計算之資料。
* window_length: _int_
        決定 n 日。

[回到選單](#menu)

In [7]:
from zipline.pipeline.factors import LinearWeightedMovingAverage
def make_pipeline():
    return Pipeline(
        columns = {
            "LWMA": LinearWeightedMovingAverage(
                inputs = [TWEquityPricing.close], 
                window_length = 10
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

LWMA
2018-02-02 00:00:00+00:00 Equity(0 [2330])      257.309091
                          Equity(1 [IR0001])  19108.238000
2018-02-05 00:00:00+00:00 Equity(0 [2330])      257.581818
                          Equity(1 [IR0001])  19095.746909
2018-02-06 00:00:00+00:00 Equity(0 [2330])      256.600000
...                                                    ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  30947.352545
2022-06-30 00:00:00+00:00 Equity(0 [2330])      494.372727
                          Equity(1 [IR0001])  30887.412364
2022-07-01 00:00:00+00:00 Equity(0 [2330])      490.636364
                          Equity(1 [IR0001])  30704.338182

[2146 rows x 1 columns]

<span id="ExponentialWeightedMovingAverage"></span>
### zipline.pipeline.factors.ExponentialWeightedMovingAverage

計算 n 日的指數加權移動平均，計算方法: 

$$
\frac{\sum_{i=1}^{n} {decay}^i \times x_{i}}{\sum_{i=1}^{n} {decay}^i}
$$

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        所欲計算之資料。
* window_length: _int_
        決定 n 日。
* decay_rate: _float_
        指數衰退率。

[回到選單](#menu)

In [8]:
from zipline.pipeline.factors import ExponentialWeightedMovingAverage
def make_pipeline():
    return Pipeline(
        columns = {
            "EWMA": ExponentialWeightedMovingAverage(
                inputs = [TWEquityPricing.close],
                window_length = 10,
                decay_rate = 0.1
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

EWMA
2018-02-02 00:00:00+00:00 Equity(0 [2330])      259.035181
                          Equity(1 [IR0001])  19111.308296
2018-02-05 00:00:00+00:00 Equity(0 [2330])      259.453518
                          Equity(1 [IR0001])  19067.749830
2018-02-06 00:00:00+00:00 Equity(0 [2330])      253.645352
...                                                    ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  31056.864215
2022-06-30 00:00:00+00:00 Equity(0 [2330])      491.648000
                          Equity(1 [IR0001])  30774.287421
2022-07-01 00:00:00+00:00 Equity(0 [2330])      477.564800
                          Equity(1 [IR0001])  30037.387742

[2146 rows x 1 columns]

<span id="ExponentialWeightedMovingStdDev"></span>
### zipline.pipeline.factors.ExponentialWeightedMovingStdDev

計算 n 日的指數加權移動標準差。
#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        所欲計算之資料。
* window_length: _int_
        決定 n 日。
* decay_rate: _float_
        指數衰退率。

[回到選單](#menu)

In [9]:
from zipline.pipeline.factors import ExponentialWeightedMovingStdDev
def make_pipeline():
    return Pipeline(
        columns = {
            "EWMSTD": ExponentialWeightedMovingStdDev(
                inputs = [TWEquityPricing.close],
                window_length = 10,
                decay_rate = 0.1
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

EWMSTD
2018-02-02 00:00:00+00:00 Equity(0 [2330])      3.307859
                          Equity(1 [IR0001])   71.847546
2018-02-05 00:00:00+00:00 Equity(0 [2330])      1.095966
                          Equity(1 [IR0001])   40.935224
2018-02-06 00:00:00+00:00 Equity(0 [2330])      4.553660
...                                                  ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  152.145031
2022-06-30 00:00:00+00:00 Equity(0 [2330])      4.637399
                          Equity(1 [IR0001])  226.079224
2022-07-01 00:00:00+00:00 Equity(0 [2330])     11.106583
                          Equity(1 [IR0001])  580.480345

[2146 rows x 1 columns]

<span id="Latest"></span>

### zipline.pipeline.factors.Latest

輸出最近一期資料。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        所欲計算之資料。

[回到選單](#menu)

In [10]:
from zipline.pipeline.factors import Latest
def make_pipeline():
    return Pipeline(
        columns = {
            "Latest": Latest(
                inputs = [TWEquityPricing.close]
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

Latest
2018-02-02 00:00:00+00:00 Equity(0 [2330])      259.50
                          Equity(1 [IR0001])  19121.21
2018-02-05 00:00:00+00:00 Equity(0 [2330])      259.50
                          Equity(1 [IR0001])  19062.91
2018-02-06 00:00:00+00:00 Equity(0 [2330])      253.00
...                                                ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  31044.99
2022-06-30 00:00:00+00:00 Equity(0 [2330])      491.00
                          Equity(1 [IR0001])  30742.89
2022-07-01 00:00:00+00:00 Equity(0 [2330])      476.00
                          Equity(1 [IR0001])  29955.51

[2146 rows x 1 columns]

<span id="MaxDrawdown"></span>

### zipline.pipeline.factors.MaxDrawdown

以 n 天為周期計算最大回撤。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算最大回徹所需要的價格資訊。
* window_lengthL: _int_
        以 n 天為週期計算。


[回到選單](#menu)

In [11]:
from zipline.pipeline.factors import MaxDrawdown
def make_pipeline():
    return Pipeline(
        columns = {
            "MaxDrawdown": MaxDrawdown(
                inputs = [TWEquityPricing.close],
                window_length = 1
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

MaxDrawdown
2018-02-02 00:00:00+00:00 Equity(0 [2330])            0.0
                          Equity(1 [IR0001])          0.0
2018-02-05 00:00:00+00:00 Equity(0 [2330])            0.0
                          Equity(1 [IR0001])          0.0
2018-02-06 00:00:00+00:00 Equity(0 [2330])            0.0
...                                                   ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])          0.0
2022-06-30 00:00:00+00:00 Equity(0 [2330])            0.0
                          Equity(1 [IR0001])          0.0
2022-07-01 00:00:00+00:00 Equity(0 [2330])            0.0
                          Equity(1 [IR0001])          0.0

[2146 rows x 1 columns]

<span id="Returns"></span>

### zipline.pipeline.factors.Returns

以 n 天為窗格，計算報酬率。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算報酬率所需要的價格資訊，預設 = EquityPricing.close。
* window_lengthL: _int_
        以 n 天為週期。


[回到選單](#menu)

In [12]:
from zipline.pipeline.factors import Returns
def make_pipeline():
    return Pipeline(
        columns = {
            "Returns": Returns(
                inputs = [TWEquityPricing.close],
                window_length = 2
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

Returns
2018-02-02 00:00:00+00:00 Equity(0 [2330])    0.017647
                          Equity(1 [IR0001])  0.005086
2018-02-05 00:00:00+00:00 Equity(0 [2330])    0.000000
                          Equity(1 [IR0001]) -0.003049
2018-02-06 00:00:00+00:00 Equity(0 [2330])   -0.025048
...                                                ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001]) -0.005615
2022-06-30 00:00:00+00:00 Equity(0 [2330])   -0.013065
                          Equity(1 [IR0001]) -0.009731
2022-07-01 00:00:00+00:00 Equity(0 [2330])   -0.030550
                          Equity(1 [IR0001]) -0.025612

[2146 rows x 1 columns]

<span id="RollingPearson"></span>

### zipline.pipeline.factors.RollingPearson

計算因子或變數之間的滾動皮爾森相關係數。

#### Parameters:
* base_factor: _zipline.pipeline.Factor_
        計算相關係數的因子。
* target: _zipline.pipeline.Term or _numerical term_
        與 base_factor 計算彼此間的相關係數。
* correlation_length : _int_
        向前 n 天為窗格計算相關係數，若僅計算前一日設定為 2。
* mask: _zipline.pipeline.Filter_
        遮蔽特定證券所需的濾網，遮蔽者將不會計算相關係數。


[回到選單](#menu)

In [13]:
from zipline.pipeline.factors import RollingPearson
from zipline.pipeline.factors import DailyReturns
from zipline.pipeline.factors import Returns
def make_pipeline():
    base_factor = DailyReturns(inputs = [TWEquityPricing.close])
    target = Returns(inputs = [TWEquityPricing.close], window_length = 6)
    return Pipeline(
        columns = {
            "RollingPearson": RollingPearson(
                base_factor = base_factor,
                target = target,
                correlation_length = 10
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

RollingPearson
2018-02-02 00:00:00+00:00 Equity(0 [2330])          0.396933
                          Equity(1 [IR0001])        0.476625
2018-02-05 00:00:00+00:00 Equity(0 [2330])          0.306692
                          Equity(1 [IR0001])        0.412337
2018-02-06 00:00:00+00:00 Equity(0 [2330])          0.235285
...                                                      ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])        0.724798
2022-06-30 00:00:00+00:00 Equity(0 [2330])          0.450101
                          Equity(1 [IR0001])        0.612251
2022-07-01 00:00:00+00:00 Equity(0 [2330])          0.520262
                          Equity(1 [IR0001])        0.519533

[2146 rows x 1 columns]

<span id="RollingLinearRegressionOfReturns"></span>

### zipline.pipeline.factors.RollingLinearRegressionOfReturns

以一個特定資產為自變數，其餘資產為應變數，進行 OLS 迴歸。會輸出迴歸式的 beta, alpha, r_value, p_value 與 standard error。迴歸式如下: 

$$
Return_{i,t} = \beta_{i} \times {Certain Return}_{i,t} + \alpha_{i}
$$

#### Parameters:
* target: _zipline.assets.Asset_
        歸類於自變數的資產。
* returns_length : _int_
        向前 n 天為窗格計算報酬，若為日報酬率，設定為 2。
* regression_length: _int_
        計算各項迴歸式的向前窗格天數。
* mask: _zipline.pipeline.Filter_
        遮蔽特定證券所需的濾網，遮蔽者將不會計算相關係數。

[回到選單](#menu)

In [14]:
from zipline.pipeline.factors import RollingLinearRegressionOfReturns
from zipline.pipeline.filters import StaticAssets
from zipline import run_algorithm
from zipline.api import symbol, attach_pipeline, pipeline_output

def make_pipeline():
    regressor = RollingLinearRegressionOfReturns(
                target = symbol("IR0001"),
                returns_length = 2,
                regression_length = 14,
            )
    return Pipeline(
        columns = {
            "beta": regressor.beta,
            "alpha": regressor.alpha,
            "r_value": regressor.r_value,
            "p_value": regressor.p_value,
            "stderr": regressor.stderr
        }, screen = ~StaticAssets([symbol("IR0001")])
    )

def initialize(context):
    my_pipe = attach_pipeline(make_pipeline(), 'my_pipe')
    
def handle_data(context, data):
    pipe = pipeline_output('my_pipe')
    print("=" * 100)
    print(f"Beta: {pipe.beta}")
    print(f"alpha: {pipe.alpha}")
    print(f"r_value: {pipe.r_value}")
    print(f"p_value: {pipe.p_value}")
    print(f"stderr: {pipe.stderr}")

def analyze(context, perf):
    pass

results = run_algorithm(
    start = start_time,
    end = end_time,
    initialize = initialize,
    capital_base = 1e6,
    handle_data = handle_data,
    analyze = analyze, 
    bundle = 'tquant'
)


Beta: Equity(0 [2330])    2.628594
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.001886
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.911471
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000006
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.342465
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    2.652884
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.002775
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.914045
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000005
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.339837
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    2.361614
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.00435
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.923077
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000003
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.284059
Name: stderr, dtype: float64
Beta: Equity(0 [2

Beta: Equity(0 [2330])    1.377859
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.003218
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.828118
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000254
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.26924
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.370142
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.002923
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.799023
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000608
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.297651
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.427362
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.002751
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.813085
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000406
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.295012
Name: stderr, dtype: float64
Beta: Equity(0 [2

Beta: Equity(0 [2330])    1.270709
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.001315
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.692781
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.006022
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.381842
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.353604
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.000204
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.819436
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000335
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.273319
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.094429
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.000509
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.77566
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.001113
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.257075
Name: stderr, dtype: float64
Beta: Equity(0 [2

Beta: Equity(0 [2330])    1.832597
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.002306
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.89548
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000015
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.262954
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.961585
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.002297
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.906303
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000008
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.26406
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.753001
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.001145
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.862957
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000071
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.2963
Name: stderr, dtype: float64
Beta: Equity(0 [2330

Beta: Equity(0 [2330])    2.274247
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.001076
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.943437
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    4.182868e-07
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.230721
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    2.234966
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.001131
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.915052
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000005
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.284381
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    2.277079
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.001042
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.924455
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000002
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.271118
Name: stderr, dtype: float64
Beta: Equity

...

Beta: Equity(0 [2330])    1.791962
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.001534
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.938773
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    6.660159e-07
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.189851
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.894803
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.001167
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.956509
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    8.893209e-08
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.166811
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.979897
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.001779
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.965953
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    2.089771e-08
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.153082
Name: stderr, dtype: float64
Beta

Beta: Equity(0 [2330])    1.771375
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.003704
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.90678
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000008
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.237751
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.760517
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.003991
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.903691
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000009
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.240803
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.762634
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.004313
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.899636
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000012
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.246961
Name: stderr, dtype: float64
Beta: Equity(0 [2

Beta: Equity(0 [2330])    0.676474
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.00119
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.779179
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.001021
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.157091
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    0.523232
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.000065
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.537513
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.047441
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.236959
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    0.720373
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.000238
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.723501
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.003447
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.198418
Name: stderr, dtype: float64
Beta: Equity(0 [2

Beta: Equity(0 [2330])    1.166957
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.002001
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.696364
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.005662
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.347187
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.174427
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.001173
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.727621
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.003181
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.319625
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.130116
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.000528
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.760719
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.001582
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.278361
Name: stderr, dtype: float64
Beta: Equity(0 [

Beta: Equity(0 [2330])    1.155928
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.001528
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.909925
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000006
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.152105
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.166394
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.000655
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.885708
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000025
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.176486
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.198162
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.000082
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.911554
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000006
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.156018
Name: stderr, dtype: float64
Beta: Equity(0 [

<span id="RollingSpearmanOfReturns"></span>

### zipline.pipeline.factors.RollingSpearmanOfReturns

給定一個特定資產，計算其他資產報酬與特定資產報酬的斯匹爾曼等級相關係數。

#### Parameters:
* target: _zipline.assets.Asset_
        特定資產。
* returns_length : _int_
        向前 n 天為窗格計算報酬，若為日報酬率，設定為 2。
* correlation_length : _int_
        計算相關係數的窗格數。
* mask: _zipline.pipeline.Filter_
        遮蔽特定證券所需的濾網，遮蔽者將不會計算相關係數。

[回到選單](#menu)

In [15]:
from zipline.pipeline.factors import RollingSpearmanOfReturns
from zipline.pipeline.filters import StaticAssets
from zipline import run_algorithm
from zipline.api import symbol, attach_pipeline, pipeline_output

def make_pipeline():
    regressor = RollingSpearmanOfReturns(
                target = symbol("IR0001"),
                returns_length = 2,
                correlation_length = 14,
            )
    return Pipeline(
        columns = {
            "RollingSpearmanOfReturns": regressor
        }, screen = ~StaticAssets([symbol("IR0001")])
    )

def initialize(context):
    my_pipe = attach_pipeline(make_pipeline(), 'my_pipe')
    
def handle_data(context, data):
    pipe = pipeline_output('my_pipe')
    print("=" * 100)
    print(pipe)

def analyze(context, perf):
    pass

results = run_algorithm(
    start = start_time,
    end = end_time,
    initialize = initialize,
    capital_base = 1e6,
    handle_data = handle_data,
    analyze = analyze, 
    bundle = 'tquant'
)


                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.859341
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.904291
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.926293
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.926293
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.860287
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.875688
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.880089
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.888889
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.915292
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.920879
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.912088
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.750275
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.745055
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.802198
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.846154
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.868132
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.863736
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.789011
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.789011
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.723077
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.687912
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.687912
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.815385
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.736264
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.679121
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.70989
                  RollingSpearmanOfReturns
Equity(0 [2330])                       0.6
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.586813
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.696703
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.630769
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.67033
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.582418
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.683516
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.864687
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.864687
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.871288
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.863438
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.894275
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.880803
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.907294
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.907294
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.894048
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.739522
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.795605
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.949342
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.959297
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.959297
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.973626
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.973626
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.942857
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.942857
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.92967
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.916484
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.925275
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.859341
                  RollingSpearmanOfReturns
Equity(0 [2

...

                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.907692
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.837363
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.78022
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.810989
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.832967
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.824176
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.859341
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.876923
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.898901
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.89011
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.885714
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.872527
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.898901
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.907692
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.907692
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.903297
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.938462
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.916484
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.864687
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.89109
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.833884
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.761277
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.730473
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.671068
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.715072
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.745875
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.705495
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.740659
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.402198
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.507692
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.318681
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.38022
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.428571
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.70989
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.714286
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.775824
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.758242
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.753846
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.727473
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.758242
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.78022
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.767033
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.841758
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.824176
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.855886
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.876923
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.89011
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.854945
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.841758
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.898901
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.934066
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.947253
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.964835
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.964835
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.951648
                  RollingSpearmanOfReturns
Equity(0 [2

<span id="SimpleBeta"></span>

### zipline.pipeline.factors.SimpleBeta

給定一個特定資產，以該特定資產為自變數，其他資產為應變數，進行迴歸計算 beta 值，與 RollingLinearRegressionOfReturns 的 beta 等價。

#### Parameters:
* target: _zipline.assets.Asset_
        特定資產。
* regression_length  : _int_
        計算 beta 的窗格數。

[回到選單](#menu)

In [16]:
from zipline.pipeline.factors import SimpleBeta
from zipline.pipeline.filters import StaticAssets
from zipline import run_algorithm
from zipline.api import symbol, attach_pipeline, pipeline_output

def make_pipeline():
    Beta = SimpleBeta(
                target = symbol("IR0001"),
                regression_length = 30
            )
    return Pipeline(
        columns = {
            "SimpleBeta": Beta
        }, screen = ~StaticAssets([symbol("IR0001")])
    )

def initialize(context):
    my_pipe = attach_pipeline(make_pipeline(), 'my_pipe')
    
def handle_data(context, data):
    pipe = pipeline_output('my_pipe')
    print("=" * 100)
    print(pipe)

def analyze(context, perf):
    pass

results = run_algorithm(
    start = start_time,
    end = end_time,
    initialize = initialize,
    capital_base = 1e6,
    handle_data = handle_data,
    analyze = analyze, 
    bundle = 'tquant'
)


                  SimpleBeta
Equity(0 [2330])    2.118428
                  SimpleBeta
Equity(0 [2330])    2.104674
                  SimpleBeta
Equity(0 [2330])    1.986685
                  SimpleBeta
Equity(0 [2330])    1.437257
                  SimpleBeta
Equity(0 [2330])    1.373916
                  SimpleBeta
Equity(0 [2330])    1.399631
                  SimpleBeta
Equity(0 [2330])    1.438815
                  SimpleBeta
Equity(0 [2330])     1.44381
                  SimpleBeta
Equity(0 [2330])     1.33807
                  SimpleBeta
Equity(0 [2330])    1.332773
                  SimpleBeta
Equity(0 [2330])    1.343458
                  SimpleBeta
Equity(0 [2330])    1.345029
                  SimpleBeta
Equity(0 [2330])     1.34321
                  SimpleBeta
Equity(0 [2330])    1.348084
                  SimpleBeta
Equity(0 [2330])    1.331513
                  SimpleBeta
Equity(0 [2330])    1.318237
                  SimpleBeta
Equity(0 [2330])    1.364871
              

                  SimpleBeta
Equity(0 [2330])    1.485963
                  SimpleBeta
Equity(0 [2330])    1.523051
                  SimpleBeta
Equity(0 [2330])    1.514718
                  SimpleBeta
Equity(0 [2330])    1.632473
                  SimpleBeta
Equity(0 [2330])    1.413863
                  SimpleBeta
Equity(0 [2330])     1.43923
                  SimpleBeta
Equity(0 [2330])    1.419692
                  SimpleBeta
Equity(0 [2330])    1.454371
                  SimpleBeta
Equity(0 [2330])     1.46046
                  SimpleBeta
Equity(0 [2330])     1.45421
                  SimpleBeta
Equity(0 [2330])    1.429712
                  SimpleBeta
Equity(0 [2330])    1.417268
                  SimpleBeta
Equity(0 [2330])    1.461774
                  SimpleBeta
Equity(0 [2330])    1.477425
                  SimpleBeta
Equity(0 [2330])    1.424718
                  SimpleBeta
Equity(0 [2330])    1.428287
                  SimpleBeta
Equity(0 [2330])    1.566816
              

                  SimpleBeta
Equity(0 [2330])    1.846634
                  SimpleBeta
Equity(0 [2330])     2.09722
                  SimpleBeta
Equity(0 [2330])      2.1164
                  SimpleBeta
Equity(0 [2330])    1.993218
                  SimpleBeta
Equity(0 [2330])    2.039016
                  SimpleBeta
Equity(0 [2330])    2.118913
                  SimpleBeta
Equity(0 [2330])     2.11638
                  SimpleBeta
Equity(0 [2330])    2.110273
                  SimpleBeta
Equity(0 [2330])     2.19959
                  SimpleBeta
Equity(0 [2330])    2.206525
                  SimpleBeta
Equity(0 [2330])    2.188291
                  SimpleBeta
Equity(0 [2330])    2.199878
                  SimpleBeta
Equity(0 [2330])    2.195078
                  SimpleBeta
Equity(0 [2330])    2.241845
                  SimpleBeta
Equity(0 [2330])    2.233158
                  SimpleBeta
Equity(0 [2330])    2.017775
                  SimpleBeta
Equity(0 [2330])    1.981517
              

                  SimpleBeta
Equity(0 [2330])      1.6064
                  SimpleBeta
Equity(0 [2330])    1.572111
                  SimpleBeta
Equity(0 [2330])     1.55954
                  SimpleBeta
Equity(0 [2330])    1.527829
                  SimpleBeta
Equity(0 [2330])    1.498956
                  SimpleBeta
Equity(0 [2330])    1.495299
                  SimpleBeta
Equity(0 [2330])    1.474353
                  SimpleBeta
Equity(0 [2330])    1.456875
                  SimpleBeta
Equity(0 [2330])    1.408465
                  SimpleBeta
Equity(0 [2330])    1.379396
                  SimpleBeta
Equity(0 [2330])     1.38582
                  SimpleBeta
Equity(0 [2330])    1.431116
                  SimpleBeta
Equity(0 [2330])    1.446451
                  SimpleBeta
Equity(0 [2330])    1.427296
                  SimpleBeta
Equity(0 [2330])    1.445101
                  SimpleBeta
Equity(0 [2330])    1.454954
                  SimpleBeta
Equity(0 [2330])    1.442527
              

                  SimpleBeta
Equity(0 [2330])    1.085642
                  SimpleBeta
Equity(0 [2330])    1.065661
                  SimpleBeta
Equity(0 [2330])    1.069688
                  SimpleBeta
Equity(0 [2330])    1.081727
                  SimpleBeta
Equity(0 [2330])    0.988514
                  SimpleBeta
Equity(0 [2330])    0.992408
                  SimpleBeta
Equity(0 [2330])    1.024007
                  SimpleBeta
Equity(0 [2330])    1.027776
                  SimpleBeta
Equity(0 [2330])    0.998042
                  SimpleBeta
Equity(0 [2330])    0.893965
                  SimpleBeta
Equity(0 [2330])    1.056986
                  SimpleBeta
Equity(0 [2330])    1.091615
                  SimpleBeta
Equity(0 [2330])    1.088038
                  SimpleBeta
Equity(0 [2330])    1.067629
                  SimpleBeta
Equity(0 [2330])    1.067791
                  SimpleBeta
Equity(0 [2330])      1.0662
                  SimpleBeta
Equity(0 [2330])    1.067859
              

                  SimpleBeta
Equity(0 [2330])    1.808596
                  SimpleBeta
Equity(0 [2330])    1.783536
                  SimpleBeta
Equity(0 [2330])    1.807424
                  SimpleBeta
Equity(0 [2330])    1.813525
                  SimpleBeta
Equity(0 [2330])    1.767718
                  SimpleBeta
Equity(0 [2330])    1.785232
                  SimpleBeta
Equity(0 [2330])    1.776286
                  SimpleBeta
Equity(0 [2330])    1.844168
                  SimpleBeta
Equity(0 [2330])    1.454266
                  SimpleBeta
Equity(0 [2330])    1.462474
                  SimpleBeta
Equity(0 [2330])    1.436494
                  SimpleBeta
Equity(0 [2330])    1.421834
                  SimpleBeta
Equity(0 [2330])    1.399218
                  SimpleBeta
Equity(0 [2330])    1.364026
                  SimpleBeta
Equity(0 [2330])    1.364642
                  SimpleBeta
Equity(0 [2330])     1.42139
                  SimpleBeta
Equity(0 [2330])    1.440532
              

                  SimpleBeta
Equity(0 [2330])    1.433857
                  SimpleBeta
Equity(0 [2330])    1.465513
                  SimpleBeta
Equity(0 [2330])    1.449751
                  SimpleBeta
Equity(0 [2330])    1.471294
                  SimpleBeta
Equity(0 [2330])    1.455475
                  SimpleBeta
Equity(0 [2330])    1.441964
                  SimpleBeta
Equity(0 [2330])    1.464589
                  SimpleBeta
Equity(0 [2330])    1.473781
                  SimpleBeta
Equity(0 [2330])    1.478557
                  SimpleBeta
Equity(0 [2330])    1.465486
                  SimpleBeta
Equity(0 [2330])    1.490116
                  SimpleBeta
Equity(0 [2330])    1.483207
                  SimpleBeta
Equity(0 [2330])     1.46037
                  SimpleBeta
Equity(0 [2330])    1.497575
                  SimpleBeta
Equity(0 [2330])    1.457735
                  SimpleBeta
Equity(0 [2330])    1.426041
                  SimpleBeta
Equity(0 [2330])    1.406702
              

                  SimpleBeta
Equity(0 [2330])    0.810054
                  SimpleBeta
Equity(0 [2330])    0.795533
                  SimpleBeta
Equity(0 [2330])    0.847537
                  SimpleBeta
Equity(0 [2330])    0.860427
                  SimpleBeta
Equity(0 [2330])    0.864501
                  SimpleBeta
Equity(0 [2330])    0.900097
                  SimpleBeta
Equity(0 [2330])    0.884539
                  SimpleBeta
Equity(0 [2330])    0.909658
                  SimpleBeta
Equity(0 [2330])    0.917505
                  SimpleBeta
Equity(0 [2330])    0.962937
                  SimpleBeta
Equity(0 [2330])    0.946497
                  SimpleBeta
Equity(0 [2330])    0.948983
                  SimpleBeta
Equity(0 [2330])    0.947249
                  SimpleBeta
Equity(0 [2330])    0.959655
                  SimpleBeta
Equity(0 [2330])    0.964385
                  SimpleBeta
Equity(0 [2330])    0.957006
                  SimpleBeta
Equity(0 [2330])    0.989552
              

                  SimpleBeta
Equity(0 [2330])    1.073686
                  SimpleBeta
Equity(0 [2330])    1.127625
                  SimpleBeta
Equity(0 [2330])     1.13378
                  SimpleBeta
Equity(0 [2330])     1.13761
                  SimpleBeta
Equity(0 [2330])    1.170437
                  SimpleBeta
Equity(0 [2330])    1.184478
                  SimpleBeta
Equity(0 [2330])    1.189153
                  SimpleBeta
Equity(0 [2330])    1.159107
                  SimpleBeta
Equity(0 [2330])    1.175846
                  SimpleBeta
Equity(0 [2330])    1.185368
                  SimpleBeta
Equity(0 [2330])    1.199736
                  SimpleBeta
Equity(0 [2330])    1.190139
                  SimpleBeta
Equity(0 [2330])    1.195912
                  SimpleBeta
Equity(0 [2330])    1.203955
                  SimpleBeta
Equity(0 [2330])    1.191572
                  SimpleBeta
Equity(0 [2330])    1.208969
                  SimpleBeta
Equity(0 [2330])    1.212773
              

<span id="RSI"></span>

### zipline.pipeline.factors.RSI

以 n 天為窗格，計算 RSI。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算報酬率所需要的價格資訊，預設 = EquityPricing.close。
* window_lengthL: _int_
        以 n 天為週期，預設為 15 日。


[回到選單](#menu)

In [17]:
from zipline.pipeline.factors import RSI
def make_pipeline():
    return Pipeline(
        columns = {
            "RSI": RSI(
                inputs = [TWEquityPricing.close],
                window_length = 87
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

RSI
2018-02-02 00:00:00+00:00 Equity(0 [2330])    60.436893
                          Equity(1 [IR0001])  60.170946
2018-02-05 00:00:00+00:00 Equity(0 [2330])    60.436893
                          Equity(1 [IR0001])  59.138719
2018-02-06 00:00:00+00:00 Equity(0 [2330])    57.793765
...                                                 ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  41.305609
2022-06-30 00:00:00+00:00 Equity(0 [2330])    40.030904
                          Equity(1 [IR0001])  41.586548
2022-07-01 00:00:00+00:00 Equity(0 [2330])    39.244412
                          Equity(1 [IR0001])  40.149446

[2146 rows x 1 columns]

<span id="VWAP"></span>

### zipline.pipeline.factors.VWAP

以 n 天為窗格，計算 Volume Weighted Average Price (以成交量加權價格)。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算報酬率所需要的價格與成交量資訊，預設 = EquityPricing.close,EquityPricing.volume。
* window_lengthL: _int_
        以 n 天為週期。


[回到選單](#menu)

In [18]:
from zipline.pipeline.factors import VWAP
def make_pipeline():
    return Pipeline(
        columns = {
            "VWAP": VWAP(
                inputs = [TWEquityPricing.close, TWEquityPricing.volume],
                window_length = 87
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

VWAP
2018-02-02 00:00:00+00:00 Equity(0 [2330])      238.161039
                          Equity(1 [IR0001])  18491.679435
2018-02-05 00:00:00+00:00 Equity(0 [2330])      238.608356
                          Equity(1 [IR0001])  18499.878408
2018-02-06 00:00:00+00:00 Equity(0 [2330])      238.922956
...                                                    ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  33489.451784
2022-06-30 00:00:00+00:00 Equity(0 [2330])      551.037441
                          Equity(1 [IR0001])  33419.285813
2022-07-01 00:00:00+00:00 Equity(0 [2330])      549.093145
                          Equity(1 [IR0001])  33317.315573

[2146 rows x 1 columns]

<span id="WeightedAverageValue"></span>

### zipline.pipeline.factors.WeightedAverageValue

以 n 天為窗格，與 VWAP 相似，計算以成交量加權的某數值平均。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算報酬率所需要的資訊，預設 = EquityPricing.close,EquityPricing.volume。
* window_lengthL: _int_
        以 n 天為週期。


[回到選單](#menu)

In [19]:
from zipline.pipeline.factors import WeightedAverageValue
def make_pipeline():
    return Pipeline(
        columns = {
            "WeightedAverageValue": WeightedAverageValue(
                inputs = [TWEquityPricing.high, TWEquityPricing.volume],
                window_length = 10
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

WeightedAverageValue
2018-02-02 00:00:00+00:00 Equity(0 [2330])              260.303304
                          Equity(1 [IR0001])              0.000000
2018-02-05 00:00:00+00:00 Equity(0 [2330])              260.971786
                          Equity(1 [IR0001])              0.000000
2018-02-06 00:00:00+00:00 Equity(0 [2330])              260.065718
...                                                            ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])              0.000000
2022-06-30 00:00:00+00:00 Equity(0 [2330])              501.895564
                          Equity(1 [IR0001])              0.000000
2022-07-01 00:00:00+00:00 Equity(0 [2330])              498.581515
                          Equity(1 [IR0001])              0.000000

[2146 rows x 1 columns]

<span id="PercentChange"></span>

### zipline.pipeline.factors.PercentChange

以 n 天為窗格，計算 x 變數百分比變化。計算公式:

$$
\frac{new - old}{|old|}
$$

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        x 變數資訊。
* window_lengthL: _int_
        以 n 天為週期，需大於等於 2。


[回到選單](#menu)

In [20]:
from zipline.pipeline.factors import PercentChange
def make_pipeline():
    return Pipeline(
        columns = {
            "PercentChange": PercentChange(
                inputs = [TWEquityPricing.close],
                window_length = 2
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

PercentChange
2018-02-02 00:00:00+00:00 Equity(0 [2330])         0.017647
                          Equity(1 [IR0001])       0.005086
2018-02-05 00:00:00+00:00 Equity(0 [2330])         0.000000
                          Equity(1 [IR0001])      -0.003049
2018-02-06 00:00:00+00:00 Equity(0 [2330])        -0.025048
...                                                     ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])      -0.005615
2022-06-30 00:00:00+00:00 Equity(0 [2330])        -0.013065
                          Equity(1 [IR0001])      -0.009731
2022-07-01 00:00:00+00:00 Equity(0 [2330])        -0.030550
                          Equity(1 [IR0001])      -0.025612

[2146 rows x 1 columns]

<span id="PeerCount"></span>

### zipline.pipeline.factors.PeerCount

以 factor/classifier 為判斷基準，呈現同類的公司數量。這裡以報酬率的四分位數為基準。

#### Parameters:
* inputs: _zipline.pipeline.factors.factor_
        判斷基準
* window_lengthL: _int_
        以 n 天為週期。


[回到選單](#menu)

In [21]:
from zipline.pipeline.factors import PeerCount, Returns
def make_pipeline():

    Ret = Returns(inputs = [TWEquityPricing.close], window_length = 2)
    quarter = Ret.quartiles()
    
    return Pipeline(
        columns = {
            "PeerCount": PeerCount(
                inputs = [quarter]
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

PeerCount
2018-02-02 00:00:00+00:00 Equity(0 [2330])          1.0
                          Equity(1 [IR0001])        1.0
2018-02-05 00:00:00+00:00 Equity(0 [2330])          1.0
                          Equity(1 [IR0001])        1.0
2018-02-06 00:00:00+00:00 Equity(0 [2330])          1.0
...                                                 ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])        1.0
2022-06-30 00:00:00+00:00 Equity(0 [2330])          1.0
                          Equity(1 [IR0001])        1.0
2022-07-01 00:00:00+00:00 Equity(0 [2330])          1.0
                          Equity(1 [IR0001])        1.0

[2146 rows x 1 columns]

<span id="RateOfChangePercentage"></span>

### zipline.pipeline.factors.RateOfChangePercentage

計算因子在指定時間長度的 %變化量，公式：(尾 - 頭) / 頭 * 100

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算所需價量資料。
* window_lengthL: _int_
        以 n 天為週期。


[回到選單](#menu)

In [22]:
from zipline.pipeline.factors import RateOfChangePercentage
def make_pipeline():
    
    return Pipeline(
        columns = {
            "RateOfChangePercentage": RateOfChangePercentage(
                inputs = [TWEquityPricing.close],
                window_length = 10
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

RateOfChangePercentage
2018-02-02 00:00:00+00:00 Equity(0 [2330])                  1.565558
                          Equity(1 [IR0001])                0.084375
2018-02-05 00:00:00+00:00 Equity(0 [2330])                 -0.764818
                          Equity(1 [IR0001])               -0.936954
2018-02-06 00:00:00+00:00 Equity(0 [2330])                 -4.887218
...                                                              ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])               -2.510118
2022-06-30 00:00:00+00:00 Equity(0 [2330])                 -3.346457
                          Equity(1 [IR0001])               -2.663798
2022-07-01 00:00:00+00:00 Equity(0 [2330])                 -4.990020
                          Equity(1 [IR0001])               -3.973330

[2146 rows x 1 columns]

<span id="Aroon"></span>

### zipline.pipeline.factors.Aroon

阿隆指標，計算從指定期間 (window_length) 股價最高/低點到現在的時間，回傳一組 (down, up)，數值介於 0 - 100。

計算方式：<br>
up = 取期間 EquityPricing.high 最高那日index，除以 (window_length - 1)，再乘上100 <br>
down = 取期間 EquityPricing.low 最低那日index，除以 (window_length - 1)，再乘上100 <br>
100 = 期間最後一日達到最高/低，0 = 期間第一日達到最高/低<br>

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算所需價量資料，預設 = EquityPricing.low, EquityPricing.high。
* window_lengthL: _int_
        以 n 天為週期。
        

#### Note:
不推薦以下使用方式。這種方式會產出帶有 tuple 格式的 DataFrame，在某些 pandas 版本（例如：1.5.3）下會出現`ValueError: no types given`的錯誤。

```python
def make_pipeline():
    
    return Pipeline(
        columns = {
            "Aroon": Aroon(
                inputs = [TWEquityPricing.high, TWEquityPricing.low],
                window_length = 10,
                mask = StaticSids([0])
            )
        }, screen=StaticSids([0])
    )
```


[回到選單](#menu)

In [23]:
from zipline.pipeline.factors import Aroon
from zipline.pipeline.filters import StaticSids
def make_pipeline():
    
    aroon = Aroon(inputs = [TWEquityPricing.high, TWEquityPricing.low],
                  window_length = 10,
                  mask = StaticSids([0]))
    
    return Pipeline(
        columns = {
            "up": aroon.up,
            "down": aroon.down,
        }, screen=StaticSids([0])
    )
run_pipeline(make_pipeline(), start_time, end_time)

up        down
2018-02-02 00:00:00+00:00 Equity(0 [2330])  22.222222    0.000000
2018-02-05 00:00:00+00:00 Equity(0 [2330])  11.111111   44.444444
2018-02-06 00:00:00+00:00 Equity(0 [2330])   0.000000  100.000000
2018-02-07 00:00:00+00:00 Equity(0 [2330])  66.666667  100.000000
2018-02-08 00:00:00+00:00 Equity(0 [2330])  55.555556   88.888889
...                                               ...         ...
2022-06-27 00:00:00+00:00 Equity(0 [2330])   0.000000  100.000000
2022-06-28 00:00:00+00:00 Equity(0 [2330])  22.222222   88.888889
2022-06-29 00:00:00+00:00 Equity(0 [2330])  11.111111   77.777778
2022-06-30 00:00:00+00:00 Equity(0 [2330])   0.000000   66.666667
2022-07-01 00:00:00+00:00 Equity(0 [2330])   0.000000  100.000000

[1073 rows x 2 columns]

<span id="FastStochasticOscillator"></span>

### zipline.pipeline.factors.FastStochasticOscillator

快速隨機指標 (K值)，0-100，通常超過 80 有超買跡象，低於 20 有超賣跡象；D 值通常為 K 值三日 SMA。

計算方式：
(最後收盤價 - 期間最低) / (期間最高 - 期間最低) * 100

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算所需價量資料，預設 = EquityPricing.close, EquityPricing.low, EquityPricing.high。
* window_lengthL: _int_
        以 n 天為週期，預設 = 14。

[回到選單](#menu)

In [24]:
from zipline.pipeline.factors import FastStochasticOscillator
def make_pipeline():
    
    return Pipeline(
        columns = {
            "FastStochasticOscillator": FastStochasticOscillator(
                inputs = [TWEquityPricing.close, TWEquityPricing.low, TWEquityPricing.high],
                window_length = 10,
                mask = StaticSids([0])
            )
        }, screen=StaticSids([0])
    )
run_pipeline(make_pipeline(), start_time, end_time)

FastStochasticOscillator
2018-02-02 00:00:00+00:00 Equity(0 [2330])                 55.172414
2018-02-05 00:00:00+00:00 Equity(0 [2330])                 53.571429
2018-02-06 00:00:00+00:00 Equity(0 [2330])                 16.129032
2018-02-07 00:00:00+00:00 Equity(0 [2330])                 16.666667
2018-02-08 00:00:00+00:00 Equity(0 [2330])                 20.000000
...                                                              ...
2022-06-27 00:00:00+00:00 Equity(0 [2330])                  4.808313
2022-06-28 00:00:00+00:00 Equity(0 [2330])                 43.548387
2022-06-29 00:00:00+00:00 Equity(0 [2330])                 40.322581
2022-06-30 00:00:00+00:00 Equity(0 [2330])                 19.354839
2022-07-01 00:00:00+00:00 Equity(0 [2330])                  0.000000

[1073 rows x 1 columns]

<span id="TrueRange"></span>

### zipline.pipeline.factors.TrueRange

真實波動幅度，從
    
    今天最高 - 今天最低
    |今天最高 - 昨天收盤| 
    |今天最低 - 昨天收盤| 
    
中挑出最大值，數值越大代表波動性越大。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算所需價量資料，預設 = EquityPricing.high, EquityPricing.low, EquityPricing.close。
* window_length: _int_
        以 n 天為週期，預設 = 2，且"目前也僅支援 window_length = 2"。

[回到選單](#menu)

In [25]:
from zipline.pipeline.factors import TrueRange
def make_pipeline():
    
    return Pipeline(
        columns = {
            "TrueRange": TrueRange(
                inputs = [TWEquityPricing.high, TWEquityPricing.low, TWEquityPricing.close],
                window_length = 2,
                mask = StaticSids([0])
            )
        }, screen=StaticSids([0])
    )
run_pipeline(make_pipeline(), start_time, end_time)

TrueRange
2018-02-02 00:00:00+00:00 Equity(0 [2330])        6.0
2018-02-05 00:00:00+00:00 Equity(0 [2330])        5.0
2018-02-06 00:00:00+00:00 Equity(0 [2330])        9.0
2018-02-07 00:00:00+00:00 Equity(0 [2330])       19.0
2018-02-08 00:00:00+00:00 Equity(0 [2330])        6.5
...                                               ...
2022-06-27 00:00:00+00:00 Equity(0 [2330])        7.0
2022-06-28 00:00:00+00:00 Equity(0 [2330])       19.5
2022-06-29 00:00:00+00:00 Equity(0 [2330])        4.0
2022-06-30 00:00:00+00:00 Equity(0 [2330])        7.5
2022-07-01 00:00:00+00:00 Equity(0 [2330])       15.0

[1073 rows x 1 columns]

<span id="IchimokuKinkoHyo"></span>

### zipline.pipeline.factors.IchimokuKinkoHyo

輸出順序為 "tenkan_sen" 轉換線， "kijun_sen" 基準線， "senkou_span_a" 先行帶A， "senkou_span_b" 先行帶B， "chikou_span" 遲行帶<br>

"tenkan_sen" 轉換線：(9日最高 ＋ 9日最低) / 2<br>
"kijun_sen" 基準線：(26日最高 ＋ 26日最低) / 2<br>
"senkou_span_a" 先行帶A：前兩項平均<br>
"senkou_span_b" 先行帶B：(52日最高 ＋ 52日最低) / 2<br>
 "chikou_span" 遲行帶：26天前收盤價<br>

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算所需價量資料，預設 = EquityPricing.high, EquityPricing.low, EquityPricing.close。
* window_lengthL: _int_
        以 n 天為週期，預設 = 52。
* tenkan_sen_length: _int_
        決定轉換線日數。
* kijun_sen_length: _int_
        決定基準線日數。
* chikou_span_length: _int_
        決定遲行帶日數。

#### Note:
不推薦以下使用方式。這種方式會產出帶有 tuple 格式的 DataFrame，在某些 pandas 版本（例如：1.5.3）下會出現`ValueError: no types given`的錯誤。

```python
def make_pipeline():
    
    return Pipeline(
        columns = {
            "IchimokuKinkoHyo": IchimokuKinkoHyo(
                inputs = [TWEquityPricing.high, TWEquityPricing.low, TWEquityPricing.close],
                window_length = 52,
                mask = StaticSids([0])
            )
        }, screen=StaticSids([0])
    )
```

[回到選單](#menu)

In [26]:
from zipline.pipeline.factors import IchimokuKinkoHyo
def make_pipeline():
    
    Ich = IchimokuKinkoHyo(
        inputs = [TWEquityPricing.high, TWEquityPricing.low, TWEquityPricing.close],
        window_length = 52,
        mask = StaticSids([0])
    )

    return Pipeline(
        columns = {
            "tenkan_sen": Ich.tenkan_sen,
            "kijun_sen": Ich.kijun_sen,
            "senkou_span_a": Ich.senkou_span_a,
            "senkou_span_b": Ich.senkou_span_b,
            "chikou_span": Ich.chikou_span
        }, screen=StaticSids([0])
    )
run_pipeline(make_pipeline(), start_time, end_time)

tenkan_sen  kijun_sen  \
2018-02-02 00:00:00+00:00 Equity(0 [2330])      259.00  245.50000   
2018-02-05 00:00:00+00:00 Equity(0 [2330])      259.00  245.50000   
2018-02-06 00:00:00+00:00 Equity(0 [2330])      257.25  246.25000   
2018-02-07 00:00:00+00:00 Equity(0 [2330])      249.00  248.50000   
2018-02-08 00:00:00+00:00 Equity(0 [2330])      247.75  249.25000   
...                                                ...        ...   
2022-06-27 00:00:00+00:00 Equity(0 [2330])      500.50  520.98723   
2022-06-28 00:00:00+00:00 Equity(0 [2330])      500.50  520.98723   
2022-06-29 00:00:00+00:00 Equity(0 [2330])      500.50  520.98723   
2022-06-30 00:00:00+00:00 Equity(0 [2330])      495.50  520.98723   
2022-07-01 00:00:00+00:00 Equity(0 [2330])      491.00  516.48723   

                                            senkou_span_a  senkou_span_b  \
2018-02-02 00:00:00+00:00 Equity(0 [2330])     252.250000     244.250000   
2018-02-05 00:00:00+00:00 Equity(0 [2330])     252.250000     244.250000   
2018-02-06 00:00:00+00:00 Equity(0 [2330])     251.750000     244.250000   
2018-02-07 00:00:00+00:00 Equity(0 [2330])     248.750000     244.250000   
2018-02-08 00:00:00+00:00 Equity(0 [2330])     248.500000     244.250000   
...                                                   ...            ...   
2022-06-27 00:00:00+00:00 Equity(0 [2330])     510.743615     529.938605   
2022-06-28 00:00:00+00:00 Equity(0 [2330])     510.743615     529.938605   
2022-06-29 00:00:00+00:00 Equity(0 [2330])     510.743615     529.938605   
2022-06-30 00:00:00+00:00 Equity(0 [2330])     508.243615     526.457515   
2022-07-01 00:00:00+00:00 Equity(0 [2330])     503.743615     521.957515   

                                            chikou_span  
2018-02-02 00:00:00+00:00 Equity(0 [2330])   225.000000  
2018-02-05 00:00:00+00:00 Equity(0 [2330])   226.000000  
2018-02-06 00:00:00+00:00 Equity(0 [2330])   229.500000  
2018-02-07 00:00:00+00:00 Equity(0 [2330])   232.500000  
2018-02-08 00:00:00+00:00 Equity(0 [2330])   237.000000  
...                                                 ...  
2022-06-27 00:00:00+00:00 Equity(0 [2330])   519.179764  
2022-06-28 00:00:00+00:00 Equity(0 [2330])   527.136542  
2022-06-29 00:00:00+00:00 Equity(0 [2330])   525.147348  
2022-06-30 00:00:00+00:00 Equity(0 [2330])   517.190570  
2022-07-01 00:00:00+00:00 Equity(0 [2330])   521.168959  

[1073 rows x 5 columns]